In [1]:
!pip install -U accelerate
!pip install PopulationLM@git+https://github.com/JesseTNRoberts/PopulationLM
!pip install git+https://github.com/JesseTNRoberts/minicons_modded

^C
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/302.6 kB ? eta -:--:--
   --- ----------------------------------- 30.7/302.6 kB 660.6 kB/s eta 0:00:01
   ------- ------------------------------- 61.4/302.6 kB 656.4 kB/s eta 0:00:01
   ------------ ------------------------- 102.4/302.6 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 302.6/302.6 kB 1.7 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
minicons 0.2.33 requires accelerate<0.25.0,>=0.24.0, but you have accelerate 0.30.1 which is incompatible.
minicons 0.2.33 requires openai<0.29.0,>=0.28.0, but you have openai 1.23.4 which is incompatible.
minicons 0.2.33 requires urllib3<2.0.0,>=1.26.7, but you have urllib3 2.2.0 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/JesseTNRoberts/PopulationLM to c:\users\kyle\appdata\local\temp\pip-install-dtigcbfr\populationlm_0c2a977e099e44a588d2037aa9980705
  Resolved https://github.com/JesseTNRoberts/PopulationLM to commit e7adde2326e644d7c94d2c7086d9561f8aefb977
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for PopulationLM: filename=PopulationLM-0.1-py3-none-any.whl size=4293 sha256=1b225aba0a74bdb879c7cce8146080442df178f8733490cdd6d8fbad009c35e6
  Stored in directory: C:\Users\Kyle\AppData\Local\Temp\pip-ephem-wheel-cache-o2389j4i\wheels\82\f4\37\696c24928fbc45ae85ab821b94287b2e67e6399f4b4da446cb
Successfully built PopulationLM


  Running command git clone --filter=blob:none --quiet https://github.com/JesseTNRoberts/PopulationLM 'C:\Users\Kyle\AppData\Local\Temp\pip-install-dtigcbfr\populationlm_0c2a977e099e44a588d2037aa9980705'


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/JesseTNRoberts/minicons_modded to c:\users\kyle\appdata\local\temp\pip-req-build-27u8mvpb
  Resolved https://github.com/JesseTNRoberts/minicons_modded to commit 2aa9e4e05fb5c2af99f10dc75cc1e2968b5eaceb
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for minicons: filename=minicons-0.1-py3-none-any.whl size=19250 sha256=9bb636fa1eb5f8ccac551530de6d464af1da9704955940994c840732e19a85e5
  Stored in directory: C:\Users\Kyle\AppData\Local\Temp\pip-ephem-wheel-cache-zlgcklmm\wheels\26\00\46\5ace39a2d4724febe6edf6dd1706251699ab0a29fb217f0dbf
Successfully built minicons


  Running command git clone --filter=blob:none --quiet https://github.com/JesseTNRoberts/minicons_modded 'C:\Users\Kyle\AppData\Local\Temp\pip-req-build-27u8mvpb'


In [2]:
import argparse
import csv
import itertools
from tqdm import tqdm
import pandas as pd
import os

import torch
from transformers import AutoModelForMaskedLM, AutoModelForCausalLM
from torch.utils.data import DataLoader

from minicons import scorer
import PopulationLM as pop

c:\ProgramData\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def run_experiment(exp_path,
                   transformer,
                   input_file,
                   prompt_col_idx,
                   stim_col_idx,
                   model_name='bert-base-uncased',
                   results_loc=None,
                   batch_size=10,
                   num_batches=-1,
                   committee_size=50,
                   debug_early_exit=False
                  ):
  dataset = []
  with open(exp_path + '/' + input_file, "r") as f:
      reader = csv.DictReader(f, delimiter='|')
      column_names = reader.fieldnames
      for row in reader:
          dataset.append(list(row.values()))

  if "/" in model_name:
      model_name = model_name.replace("/", "_")

  if results_loc is None:
      results_loc = exp_path + f"/results_{model_name}.csv"

  results = []
  control_results = []
  conclusion_only = []

  column_names += ["stimulus_prob"]
  with open(results_loc, "w", newline='') as f:
      writer = csv.writer(f, delimiter='|')
      writer.writerow(column_names)

  # create a lambda function alias for the method that performs classifications
  call_me = lambda p1, q1: transformer.conditional_score(p1, q1, reduction=lambda x: (x.mean(0).item()))

  stimuli_loader = DataLoader(dataset, batch_size = batch_size, num_workers=0)
  if num_batches < 0:
      num_batches = len(stimuli_loader)
  for batch in tqdm(stimuli_loader):
      out_dataset = [[] for _ in range(len(batch))]
      stim_scores = []
      for i in range(len(batch)):
          out_dataset[i].extend(batch[i])

      results = {'stimulus_prob': []}
      p_list = list(batch[prompt_col_idx])
      s_list = list(batch[stim_col_idx])

      population = pop.generate_dropout_population(transformer.model, lambda: call_me(p_list, s_list), committee_size=committee_size)
      outs = [item for item in pop.call_function_with_population(transformer.model, population, lambda: call_me(p_list, s_list))]

      transposed_outs = [[row[i] for row in outs] for i in range(len(outs[0]))]

      stim_scores = [score for score in transposed_outs]

      results['stimulus_prob'].extend(stim_scores)

      out_dataset.append(results['stimulus_prob'])
      with open(results_loc, "a", newline='') as f:
          writer = csv.writer(f, delimiter='|')
          writer.writerows(list(zip(*out_dataset)))

      if debug_early_exit:
          break

  del population

  print('\nResults saved to: ', results_loc)

In [5]:
def run_all_experiments(model,
                        lm_type,
                        input_file,
                        prompt_col_idx,
                        stim_col_idx,
                        batch_size=10,
                        committee_size=50,
                        drive_loc='/content/drive/MyDrive/Research/Alignment_Testing/prompts/',
                        save_name=None,
                        token=None,
                        debug_early_exit=False):


    base_model_name = model
    device='cuda'

    # Load the model
    if lm_type == "masked" or lm_type == "mlm":
        try:
          transformer = scorer.MaskedLMScorer(base_model_name,
                                              device=device,
                                              local_files_only=False,
                                              low_cpu_mem_usage=True,
                                              torch_dtype=torch.float16,
                                              device_map="auto",
                                              token=token)
        except:
          transformer = scorer.MaskedLMScorer(base_model_name, device=device, token=token)
    elif lm_type == "incremental" or lm_type == "causal":
        try:
          transformer = scorer.IncrementalLMScorer(base_model_name,
                                                   device=device,
                                                   local_files_only=False,
                                                   low_cpu_mem_usage=True,
                                                   torch_dtype=torch.float16,
                                                   device_map="auto",
                                                   token=token)
        except:
          transformer = scorer.IncrementalLMScorer(base_model_name, device=device, token=token)

    #Overwrite local model with base model (handles local loading limitation in minicons)
    if save_name is not None:
        model_name = save_name
    else:
        model_name = base_model_name

    # if falcon:
    #     add_falcon_dropout_layers(transformer.model, MLP_layer_names=['FalconDecoderLayer'])
    # elif add_dropout:
    pop.DropoutUtils.add_new_dropout_layers(transformer.model)

    # convert the internal model to use MC Dropout
    # if not falcon:
    #   pop.DropoutUtils.convert_dropouts(transformer.model)
    pop.DropoutUtils.activate_mc_dropout(transformer.model, activate=True, random=0.1)

    for exp in experiments:
        print(f'Running experiment: {exp}')
        run_experiment(drive_loc + exp,
                       transformer,
                       input_file,
                       prompt_col_idx,
                       stim_col_idx,
                       model_name=model_name,
                       batch_size=batch_size,
                       committee_size=committee_size,
                       debug_early_exit=debug_early_exit
                       )

    del transformer
    # torch.cuda.empty_cache()

In [6]:
from google.colab import userdata
my_token = userdata.get('hf_token')

In [12]:
#Replace with experiment base directory
drive_loc='./data/'

#Name of the file where prompts are stored. Assumed to be a csv file with '|' delimiters
input_file = 'prompts.csv'

#Each experiment name should be the name a directory that descends from drive_loc
experiments = [
                'Random',
                'And_True'
              ]

#Each model should be defined using the format (model_name, model_type, save_name)
#model_name should be the name used to load the model using the transformers library (i.e. either a file location or the huggingface name)
#model_type should be one of 'masked' or 'incremental', depending on the model structure
#save_name is where the results will be the name used when creating the results file. Results file name will be <drive_loc>/<experiment_name>/results_<save_name>.csv
models = [
    ('bert-base-uncased',                   'masked',      'BERT',                ),
    # ('meta-llama/Llama-2-7b-hf',            'incremental', 'LLaMa2-7B',           ),
    # ('meta-llama/Meta-Llama-3-8B',          'incremental', 'LLaMa3-8B',           ),
    # ('meta-llama/Meta-Llama-3-8B-Instruct', 'incremental', 'LLaMa3-8B-Instruct',  ),
    # ('openai-community/gpt2',               'incremental', 'GPT2',                ),
    # ('mistralai/Mistral-7B-v0.1',           'incremental', 'Mistral-7B',          ),
    # ('openlm-research/open_llama_7b',       'incremental', 'OpenLLaMa',           ),
    # ('PKU-Alignment/beaver-7b-v1.0',        'incremental', 'Beaver-7B',           ),
    # ('PKU-Alignment/alpaca-7b-reproduced',  'incremental', 'Alpaca-7B',           ),
    # ('tiiuae/falcon-7b',                    'incremental', 'Falcon-7B',           ),
    # ('tiiuae/falcon-7b-instruct',           'incremental', 'Falcon-7B-instruct',  ),
    # ('FacebookAI/roberta-base',             'masked',      'RoBERTa',             ),
    # ('FacebookAI/roberta-large',            'masked',      'RoBERTa-large',       ),
    # ('upstage/SOLAR-10.7B-v1.0',            'incremental', 'SOLAR-10.7B',         ),
]

#These are the zero-based index of the prompt and the stimulus in the input csv file
prompt_col_index = 0
stim_col_index = 5

#Forces above cells to only run one iteration for testing purposes. Set to False when running full experiment
debug_EE = True

# Handles my_token undefined. If you want to use a huggingface token (necessary
# for LLaMa models among others), run the cell above
try:
  my_token
except NameError:
  my_token = None

In [13]:
# format: (model_name, model_type, save_name)
for mn, mt, sn in models:
    run_all_experiments(mn, mt,
                        input_file,
                        prompt_col_index,
                        stim_col_index,
                        batch_size = 24,
                        save_name = sn,
                        drive_loc= drive_loc,
                        token=my_token,
                        debug_early_exit=debug_EE)

Running experiment: Random


FileNotFoundError: [Errno 2] No such file or directory: './data/Random/anderson_true.csv'